In [25]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [26]:
df = pd.read_csv('website_classification.csv')
df

,website_url,cleaned_website_text,Category
0,https://www.booking.com/index.html?aid=1743217,official site good hotel accommodation big sav...,Travel
1,https://travelsites.com/expedia/,expedia hotel book sites like use vacation wor...,Travel
2,https://travelsites.com/tripadvisor/,tripadvisor hotel book sites like previously d...,Travel
3,https://www.momondo.in/?ispredir=true,cheap flights search compare flights momondo f...,Travel
4,https://www.ebookers.com/?AFFCID=EBOOKERS-UK.n...,bot create free account create free account si...,Travel
...,...,...,...
1441,https://www.reddit.com/r/porn/,reddit dive into anythingpornpress j to jump t...,Adult
1442,https://www.eurogirlsescort.com/#,escort directory vip escort girls escort liste...,Adult
1443,https://www.adultlook.com/,escorts escort reviews adultlooktype few keywo...,Adult
1444,https://adultwork.com/,adultworkcom adult service providers erotic co...,Adult


In [27]:
class ScrapTool:
    def visit_url(self,website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        content = requests.get(website_url,timeout=60).content
        soup = BeautifulSoup(content,'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                                               self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

In [28]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r' +',' ',text)
    return text

In [29]:
def content_generation(website):
    scrapTool = ScrapTool()
    try:
        web = dict(scrapTool.visit_url(website))
        text = cleaning_text(web['website_text'])
        return text
    except:
        print('Error: ',website)
        return None

In [31]:
websites= [
    "https://coinmarketcap.com",
    "https://coingecko.com",
    "https://coinbase.com",
    "https://binance.com",
    "https://kraken.com",
    "https://blockchain.com",
    "https://bitstamp.net",
    "https://bitfinex.com",
    "https://bittrex.com",
    "https://poloniex.com",
    "https://hitbtc.com",
    "https://kucoin.com",
    "https://bitpay.com",
    "https://ledger.com",
    "https://mycelium.com",
    "https://breadwallet.com",
    "https://electrum.org",
    "https://trustwallet.com",
    "https://coinomi.com",
    "https://abra.com",
    "https://uphold.com",
    "https://paxful.com",
    "https://localbitcoins.com",
    "https://bisq.network",
    "https://changelly.com",
    "https://coinmixer.se",
    "https://coindesk.com",
    "https://cointelegraph.com",
    "https://bitcoin.com",
    "https://news.bitcoin.com",
    "https://bitcoinmagazine.com",
    "https://newsbtc.com",
    "https://cryptoslate.com",
    "https://cryptonews.com",
    "https://cryptocoinsnews.com",
    "https://bitcoinist.com",
    "https://bitcoin.org",
    "https://greenaddress.it",
    "https://edge.app",
    "https://coinjar.com",
    "https://coinsbank.com",
    "https://keepkey.com",
    "https://bitgo.com",
    "https://xapo.com",
    "https://bitwala.com",
    "https://coincheck.com",
    "https://bitso.com",
    "https://luno.com",
    "https://bitmex.com",
    "https://deribit.com",
    "https://primexbt.com",
    "https://bitflyer.com",
    "https://bitbuy.ca",
    "https://gemini.com",
    "https://okex.com",
    "https://coinbene.com",
    "https://bitmart.com",
]

for website in websites:
    text = content_generation(website)
    if text is not None:
        web = [website,text,'Cryptocurrency']
        df.loc[len(df)] = web

Error:  https://bitstamp.net
Error:  https://trezor.io
Error:  https://airbitz.co
Error:  https://bybit.com
Error:  https://itbit.com
Error:  https://huobi.com


In [35]:
pd.set_option('display.max_rows', 100)
df.tail(94)

,website_url,cleaned_website_text,Category
1408,https://crypto.news/,crypto news latest cryptocurrency newsstay upt...,Cryptocurrency
1409,https://coindoo.com/,coindoo guides reviews and blockchain technolo...,Cryptocurrency
1410,https://smartoptions.io/,forbidden forbiddennginx,Cryptocurrency
1411,https://cryptonews.net/,world news about cryptocurrency and blockchain...,Cryptocurrency
1412,https://e-cryptonews.com/,just a momentenable javascript and cookies to ...,Cryptocurrency
1413,https://bitcoinik.com/,bitcoinik next generation bitcoin and cryptocu...,Cryptocurrency
1414,https://thecryptobasic.com/,the crypto basic latest crypto news defi topic...,Cryptocurrency
1415,https://zerocrypted.com/,zerocrypted your daily cryptocurrency news gui...,Cryptocurrency
1416,https://www.bydfi.com/en-US,bydfi,Cryptocurrency
1417,https://coingate.com/,top bitcoin and other crypto payment processor...,Cryptocurrency


In [38]:
df.drop([1463],inplace=True)
df.reset_index(drop=True,inplace=True)

In [24]:
df.to_csv('website_classification.csv',index=False)